<a href="https://colab.research.google.com/github/janbanot/msc-cs-code/blob/main/sem3/DL/DL_2025_Lab6-a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uczenie nienadzorowane -- wybrane metody

## Sieci typu koder--dekoder

In [ ]:
%%html
<!-- Potrzebne dla poprawnego wyświetlania paska postępu tqdm w VSCode https://stackoverflow.com/a/77566731 -->
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

In [ ]:
!uv pip -q install torchinfo

# Import potrzebnych modułów i funkcji

from collections import defaultdict
from random import random

import numpy as np
import torch
import torch.functional as F
from torch import nn, tensor
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Subset
import random
from torchvision import datasets, transforms, models
from PIL import Image

# Importy do wizualizacji
import matplotlib.pyplot as plt
from matplotlib import cm
from tqdm.notebook import tqdm

from torchinfo import summary

In [ ]:
def get_device():   # Obliczenia wykonamy na GPU, jeżeli jest dostępne, a na CPU w przeciwny razie
  return torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def train_unsupervised(model, train_loader, test_loader,
                optimizer,
                n_epochs=20, eval_every=1,
                loss_fn=None,
                scheduler=None,
                device=None,
                history=None):
    device = device or get_device()

    history = history or defaultdict(list)
    # Przenieś model na określone urządzenie
    model.to(device)

    # Definiowanie funkcji straty
    compute_loss = loss_fn or nn.CrossEntropyLoss()

    test_loss = history['test_loss'][-1] if history['test_loss'] else 0

    # Pętla treningowa
    for epoch in range(n_epochs):
        model.train()  # Ustaw model w tryb treningowy

        total_loss = 0
        total_samples = 0

        # Iteracja po danych treningowych
        for x_batch, _ in tqdm(train_loader):
            # Przenieś dane na określone urządzenie
            x_batch = x_batch.to(device)

            optimizer.zero_grad()  # Wyzerowanie gradientów przed kolejną iteracją
            out = model(x_batch)  # Faza w przód

            loss = compute_loss(out, x_batch)

            loss.backward()   # Faza wsteczna do obliczenia gradientów
            optimizer.step()  # Aktualizacja parametrów modelu

            total_samples += x_batch.shape[0]
            total_loss += loss.item() * x_batch.shape[0]

        train_loss = total_loss / total_samples

        if (epoch + 1) % eval_every == 0:  # Ewaluacja na zbiorze testowym
            model.eval()   # Ustaw model w tryb ewaluacji
            total_loss = 0
            total_samples = 0
            with torch.no_grad():  # Wyłączenie obliczania gradientów
                for x_batch, _ in tqdm(test_loader):
                    # Przenieś dane na określone urządzenie
                    x_batch = x_batch.to(device)

                    out = model(x_batch)  # Faza w przód

                    loss = compute_loss(out, x_batch)

                    total_samples += x_batch.shape[0]
                    total_loss += loss.item() * x_batch.shape[0]

            test_loss = total_loss / total_samples

            print(f'Epoch: {epoch}\tTrain loss: {train_loss:.3f}'\
                  f'\tTest loss: {test_loss:.3f}')
        else:
            print(f'Epoch: {epoch}\tTrain loss: {train_loss:.3f}')

        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)

        if scheduler:
            scheduler.step()

    return history

In [ ]:
class Autoencoder(nn.Module):
    """ Prosty model typu koder-dekoder """
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        # Koder (ang. encoder)
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim),
            nn.ReLU()
        )
        # Dekoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim),
            nn.Sigmoid()  # Wyj. z zakresu [0, 1]
        )

    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

In [ ]:
# Model przetwarza spłaszczone dane, czyli wektory, stąd konieczna jest transformacja
flatten_transform = transforms.Lambda(lambda x: x.view(-1))

# Transformacje, można również dodać odbicia itp
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts images to PyTorch tensors (values between 0 and 1)
    flatten_transform
])

train_dataset = mnist_train = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = mnist_test = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
torch.manual_seed(123)
device = get_device()

# Hiperparametry
input_dim = 28 * 28   # 784 for 28x28 images
latent_dim = 16       # Wymiar przestrzeni ukrytej
learning_rate = 1e-3

model = Autoencoder(input_dim, latent_dim).to(device)
print(summary(model))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=5)

train_unsupervised(model,
                   train_loader=train_loader, test_loader=test_loader,
                   optimizer=optimizer, scheduler=scheduler,
                   n_epochs=7, loss_fn=nn.BCELoss())

In [ ]:
# Ewaluacja na zb. testowym
model.eval()
with torch.no_grad():
    all_encoded = []
    all_decoded = []
    all_original = []
    for data, _ in test_loader:
        data = data.view(-1, input_dim).to(device)
        encoded = model.encode(data)
        decoded = model.decode(encoded)
        all_encoded.append(encoded.cpu())
        all_decoded.append(decoded.cpu())
        all_original.append(data.cpu())

    decoded_imgs = torch.cat(all_decoded)
    original_imgs = torch.cat(all_original)

decoded_imgs = decoded_imgs.numpy()
original_imgs = original_imgs.numpy()

# Wizualizacja
n_samples = 10
plt.figure(figsize=(20, 4))
for i in range(n_samples):
    # Oryginalne obrazy
    ax = plt.subplot(2, n_samples, i + 1)
    plt.imshow(original_imgs[i].reshape(28, 28), cmap='gray')
    plt.title("Oryginalne")
    plt.axis('off')

    # Rekonstrukcje
    ax = plt.subplot(2, n_samples, i + 1 + n_samples)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.title("Rekonstrukcje")
    plt.axis('off')
plt.show()

# Zadanie 1

Proszę sprawdzić jak zmienia się jakość rekonstrukcji w zależności od wymiarowości reprezentacji ukrytej w autokoderze
dla FashionMNIST (mogą być wybrane klasy).

Można porównać zarówno "surowe" wartości f. straty, jak i ocenić różnice wizualnie.

In [ ]:
# Transformacje (bez zmian)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Załadowanie FashionMNIST
train_fmnist = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_fmnist = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_fmnist, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_fmnist, batch_size=64, shuffle=False)

In [ ]:
latent_dims = [2, 8, 32, 128]
results = {}
reconstructions = {}

input_dim = 28 * 28
n_epochs = 5

for d in latent_dims:
    print(f"\n--- Trening dla latent_dim = {d} ---")
    model = Autoencoder(input_dim, d).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Trening
    history = train_unsupervised(
        model, train_loader, test_loader,
        optimizer, n_epochs=n_epochs, loss_fn=nn.BCELoss(), device=device
    )

    # Zapisanie wyników
    results[d] = history['test_loss'][-1]

    # Pobranie przykładowych rekonstrukcji do wizualizacji
    model.eval()
    with torch.no_grad():
        data, _ = next(iter(test_loader))
        data = data.to(device)
        recon = model(data)
        reconstructions[d] = recon.cpu().numpy()
        original_samples = data.cpu().numpy()

In [ ]:
n_samples = 5
plt.figure(figsize=(15, 2 * (len(latent_dims) + 1)))

# Oryginały
for i in range(n_samples):
    plt.subplot(len(latent_dims) + 1, n_samples, i + 1)
    plt.imshow(original_samples[i].reshape(28, 28), cmap='gray')
    if i == 0: plt.ylabel("Oryginał")
    plt.axis('off')

# Rekonstrukcje dla różnych wymiarów
for row, d in enumerate(latent_dims):
    for i in range(n_samples):
        plt.subplot(len(latent_dims) + 1, n_samples, (row + 1) * n_samples + i + 1)
        plt.imshow(reconstructions[d][i].reshape(28, 28), cmap='gray')
        if i == 0: plt.ylabel(f"Dim: {d}")
        plt.axis('off')

plt.tight_layout()
plt.show()